## 5 - Multi-lcass Sentiment Analysis

> Ở các notebooks trước, chúng ta đã được hiện phân tích cảm xúc trên bộ dữ liệu chỉ có 2 nhãn là: tích cực (pos) và tiêu cực (neg). Khi làm việc với bài toán 2 nhãn, đầu ra chỉ là 1 đại lượng vô hướng nằm trong khoảng [0, 1], biểu thị xác suất đối tượng thuộc class nào. khi có nhiều hơn 2 nhãn, đầu ra sẽ là một vector C chiều, với C là số lượng class.

> Trong notebook này, chúng ta sẽ thực hiện phân loại trên bộ dữ liệu có 6 classes. Lưu ý bộ dữ liệu này không phải là bộ dữ liệu phân tích cảm xúc, đó là bộ dữ liệu về phân loại câu hỏi: **Phân loại xem câu hỏi thuộc loại nào?**.

### Data Preprocessing

> Đầu tiên, ta thực hiện xây dựng và xử lý dữ liệu.
>> Điểm khác biệt đầu tiên so với bài trước là: ta không cần thiết lập `dtype` trong trường `LABEL`. Khi thực hiện các bài toán multi-class, PyTorch mong muốn labels phải được số hóa, có dạng `LongTensor`s. <br>
>> Điểm khác biệt thứ 2 là: ta sử dụng `TREC` để tải tập dữ liệu TREC. Đối số `fine_grained` sẽ kiểm soát số lượng nhãn của bài toán, nếu là `True`, ta sẽ sử dụng 50 classes, nếu là `False` ta sẽ sử dụng 6 classes.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data
from torchtext.legacy import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language='en_core_web_sm')

LABEL = data.LabelField()

train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

> Cùng tìm hiểu về dữ liệu nào!

In [2]:
vars(train_data.examples[-1])

{'text': ['What', 'is', 'a', 'Cartesian', 'Diver', '?'], 'label': 'DESC'}

> Tiếp theo, chúng ta xây dựng vocab. Do dataset này khá nhỏ (xấp xỉ 3800 training examples) và có số lượng tokens trong vocab cũng khá nhỏ (xấp xỉ 7500 unique tokens) nên ta không cần thiết lập tham số `max_size` trong vocab như các ví dụ trước:

In [3]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(   train_data,
                    vectors = "glove.6B.100d",              # use pre-trained GloVe vectors
                    unk_init = torch.Tensor.normal_)        # initialize unknown words with mean=0 and std=1 (phân phối chuẩn tắc
                    
                    
LABEL.build_vocab(train_data)

> Kiểm tra các labels

> Khi ta sử dụng tập dữ liệu nhỏ, bài toán có 6 classes: <br>
>> * `HUM` tương ứng với câu hỏi do con người đặt.
>> * `ENTY` là câu hỏi về thực thể.
>> * `DESC` là câu hỏi yêu cầu mô tả.
>> * `NUM` là câu hỏi yêu cầu câu trả lời là số.
>> * `LOC` là câu hỏi yêu cầu câu trả lời là vị trí.
>> * `ABBR` là câu hỏi về từ viết tắt.

In [4]:
LABEL.vocab.stoi

defaultdict(None,
            {'HUM': 0, 'ENTY': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})

> Tiếp theo, ta tạo iterators như mọi khi

In [5]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
                                                (train_data, valid_data, test_data),  # we pass in the datasets we want the iterator to draw data from
                                                batch_size = BATCH_SIZE,              # batch size
                                                sort_key=lambda x: len(x.text),       # sort by text length
                                                sort_within_batch=True,
                                                device = device)                      # on what device the iterator should load the batches

### Build the Model

> Ta sẽ sử dụng model CNN tương tự như notebook trước, tuy nhiên các model trong các notebook số 1, 2, 3 cũng có thể làm việc trên tập dữ liệu này. Điểm khác biệt là `output_dim` bây giờ là **C** thay vì 1 như trước.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNs(nn.Module):
    def __init__(self, input_dim, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)

        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1,
                                                out_channels = n_filters,
                                                kernel_size = (fs, embedding_dim)) for fs in filter_sizes
                                    ])

        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

    def forward(self, text):
            
            #text = [sent len, batch size]
    
            text = text.permute(1, 0)
    
            #text = [batch size, sent len]
    
            embedded = self.embedding(text)
    
            #embedded = [batch size, sent len, emb dim]
    
            embedded = embedded.unsqueeze(1)
    
            #embedded = [batch size, 1, sent len, emb dim]
    
            conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
    
            #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
    
            pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
    
            #pooled_n = [batch size, n_filters]
    
            cat = self.dropout(torch.cat(pooled, dim = 1))
    
            #cat = [batch size, n_filters * len(filter_sizes)]
    
            return self.fc(cat)

> Ta define model, ddamr baor rằng `OUPUT_DIM` = **C**. Ta có thể lấy **C** bằng các lấy kích thước của `LABEL` vocab, tương tự như dùng chiều dài của `TEXT` vocab để lấy kích thước từ điển đầu vào. <br>
> Do dataset ta dùng ở bài này có kích thước nhỏ hơn IDBM nên ta dùng filters có kích thước nhỏ hơn.

In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNNs(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

> Kiểm tra số lượng tham số, ta sẽ thấy do dùng filters có kích thước nhỏ hơn nên số lượng tham số cũng ít hơn.

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 842,406 trainable parameters


> Load pre-trained vectors

In [9]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [ 0.1638,  0.6046,  1.0789,  ..., -0.3140,  0.1844,  0.3624],
        ...,
        [-0.3110, -0.3398,  1.0308,  ...,  0.5317,  0.2836, -0.0640],
        [ 0.0091,  0.2810,  0.7356,  ..., -0.7508,  0.8967, -0.7631],
        [ 0.4306,  1.2011,  0.0873,  ...,  0.8817,  0.3722,  0.3458]])

> Khởi tạo các từ không có trong từ điển của pre-trained vectors và padding tokens bằng tensor 0

In [10]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)

### Train the Model

> Ở các notebooks trước, do có 2 nhãn nên ta dùng `BCEWithLogitsLoss`. Tuy nhiên ở notebook này, bài toán là *multi-class classification* nên ta sử dụng `CrossEntropyloss`.

In [11]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
critertion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = critertion.to(device)

> Tiếp theo, ta sử dụng `argmax` để lấy được địa chỉ của giá trị lớn nhất của mỗi phần tử trong batch, sau đó ta đếm số lần giá trị nhãn dự đoán bằng giá trị nhãn thực tế, cuối cùng lấy trung bình của batch.

In [22]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True)
    correct = max_preds.eq(y.view_as(max_preds)).sum()
    return correct.float() / torch.FloatTensor([y.shape[0]])

> Quy trình training tương tự như các notebooks trước, có 1 chút khác biệt là ta cần `squeeze` dự đoán của model do `CrossEntropyloss` yêu cầu đầu vào có size **[batch size, n classes]** và label có size **[batch size]**.

> Lưu ý là label phải có dạng `LongTensor`.

In [23]:
def train_model(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        predictions = model(batch.text)

        loss = criterion(predictions, batch.label)
        acc = categorical_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

> Hàm đánh giá

In [24]:
def evaluate_model(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)

            loss = criterion(predictions, batch.label)
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_model(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 0.084 | Train Acc: 98.26%
	 Val. Loss: 0.427 |  Val. Acc: 85.13%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.065 | Train Acc: 98.67%
	 Val. Loss: 0.428 |  Val. Acc: 85.58%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.054 | Train Acc: 98.85%
	 Val. Loss: 0.432 |  Val. Acc: 85.12%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.045 | Train Acc: 99.19%
	 Val. Loss: 0.432 |  Val. Acc: 86.00%
Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 0.039 | Train Acc: 99.40%
	 Val. Loss: 0.432 |  Val. Acc: 86.81%


In [27]:
model.load_state_dict(torch.load('./models/tut5-model.pt'))

test_loss, test_acc = evaluate_model(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.324 | Test Acc: 88.46%


> Hàm dự đoán tương tự như các notebooks trước, khác biệt nho nhỏ là ta sử dụng `argmax` để lấy địa chỉ của giá trị dự đoán cao nhất. Sau đó ta dùng địa chỉ này để chuyển về dạng người cho dễ nhận biết.

In [28]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += [''] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [29]:
pred_class = predict_class(model, "Who is Keyser Söze?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = HUM


In [30]:
pred_class = predict_class(model, "How many minutes are in six hundred and eighteen hours?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 3 = NUM


In [31]:
pred_class = predict_class(model, "What continent is Bulgaria in?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 4 = LOC


In [32]:
pred_class = predict_class(model, "What does WYSIWYG stand for?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 5 = ABBR
